<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
دردسر سردبیر
</font>
</h1>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
هدف از این سوال توسعه‌ی یک مدل یادگیری ماشین است که قادر به پیش‌بینی دقیق نویسنده‌ی یک متن تنها بر اساس وجود یا عدم وجود کلمات خاص است.
</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
یک مجموعه داده‌ی ساختاریافته به شما ارائه شده است که در آن هر ردیف نشان دهنده‌ی یک متن است و ستون ها یا وجود کلمات خاص در متن یا نویسنده‌ی متن را نشان می‌دهند.</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
ستون‌های کلمات: این ستون ها بیانگر کلمات جداگانه هستند. هر ورودی در این ستون‌ها باینری است که 1 نشان دهنده‌ی وجود کلمه در متن و 0 نشان دهنده‌ی عدم وجود آن است.
<br>
ستون <code>author</code>: این ستون حاوی نام نویسنده‌ای است که متن را نوشته است. این ستون، متغیر هدفی است که مدل شما باید آن را پیش‌بینی کند.</font>
</p>

<center>
<div  style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|word_1|word_2|...|word_n|author|
|:------:|:---:|:---:|:---:|:---:|
|0|1|...|1|Mason Reed|
|1|1|...|1|Ava Thompson|
|0|1|...|0|Liam Carter|

</font>
</div>
</center>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    مجموعه داده آزمایش نیز مانند مجموعه آموزش است با این تفاوت که ستون <code>author</code> که متغیر هدف مسئله است را در خود ندارد. مجموعه داده آزمایش ۲۷۶۵ سطر دارد.
</font>
</p>


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در ابتدا نیاز است فایل‌های مجموعه‌داده را بخوانید. نمونه‌های آموزشی در فایل <code>train.csv</code> و نمونه‌های آزمون که باید مقدار متغیر هدف آن‌ها را پیش‌بینی کنید در فایل <code>test.csv</code> ذخیره شده‌اند. اگر لازم دانستید می‌توانید به دلخواه خود بخشی از مجموعه‌ی آموزش را به عنوان مجموعه‌ی اعتبارسنجی نیز جدا کنید.
</font>
</p>

In [61]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_selection import SelectKBest, chi2, RFE

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import xgboost as xgb

from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV

from sklearn.metrics import f1_score, classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [2]:
train_path = '/content/drive/My Drive/Olympics - datasets/Q2/train.csv'
train = pd.read_csv('train.csv')
test_path = '/content/drive/My Drive/Olympics - datasets/Q2/test.csv'
test = pd.read_csv('test.csv')

In [3]:
print(train['author'].value_counts())


Mason Reed        653
Ava Thompson      650
Olivia Bennett    643
Ethan Brooks      627
Liam Carter       623
Name: author, dtype: int64


so it isnt imbalance and doesnt need to oversample

In [4]:
train.shape

(3196, 381)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
        در این سوال شما می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی دلخواهتان، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم مورد ارزیابی توسط سامانه داوری قرار <b>نمی‌گیرند.</b> بلکه همه آن‌ها در دقت مدل شما تاثیر خواهند گذاشت؛ بنابراین هر چه پیش‌پردازش/مهندسی ویژگی بهتری انجام دهید تا دقت مدل بهبود پیدا کند، امتیاز بیشتری از این سوال کسب خواهید کرد.

</font>
</p>

In [5]:
label_encoder = LabelEncoder()
train['author_encoded'] = label_encoder.fit_transform(train['author'])
X = train.drop(columns=['author', 'author_encoded'])
y = train['author_encoded']

X_test = test.copy()


# feature selection

In [6]:
k = 200  
selector = SelectKBest(score_func=chi2, k=k)
selector.fit(X, y)

cols = selector.get_support(indices=True)
X_selected = X.iloc[:, cols]
X_test_selected = X_test.iloc[:, cols]

print(f"Selected top {k} features.")

Selected top 200 features.


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش مدل
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    حال که داده را پاکسازی کرده و احتمالا ویژگی‌هایی را به آن افزوده یا از آن حذف کرده‌اید، وقت آن است که مدلی آموزش دهید که بتواند متغیر هدف این مسئله را پیش‌بینی کند.
</font>
</p>

In [64]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y)


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
   ارسال‌ها بر اساس امتیاز <code>F1</code> ارزیابی می‌شوند و مدل میانگین‌گیری <code>macro</code> است. امتیاز <code>F1</code> معیاری برای ارزیابی مدل است که هم بازیابی (<code>Recall</code>) و هم صحت (<code>Accuracy</code>) را در نظر می‌گیرد.
   نتیجه‌ی نهایی بر اساس فرمول زیر محاسبه می‌گردد:

$$score= round(f1score, 3) \times 100$$
</font>
</p>
<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
مقدار <code>F1 Score</code> مدل شما تا ۳ رقم اعشار گرد شده و پس از ضرب در ۱۰۰ به‌عنوان امتیاز شما از این سوال لحاظ می‌شود. بیشترین امتیاز ممکن از این سوال ۱۰۰ و کمترین امتیاز قابل قبول، ۴۰ می‌باشد. امتیاز <code>F1</code> کمتر از ۴۰ معادل صفر در نظر گرفته خواهد شد.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی مجموعه‌ی آموزش یا اعتبارسنجی ارزیابی کنید.
</font>
</p>

### Naive Bayes

In [65]:
nb_model = MultinomialNB()

nb_model.fit(X_train, y_train)

y_val_pred_nb = nb_model.predict(X_val)
f1_nb = f1_score(y_val, y_val_pred_nb, average='macro')
print(f"Validation F1 Score with Multinomial Naive Bayes: {f1_nb}")

Validation F1 Score with Multinomial Naive Bayes: 0.7541945056345464


### Logestic Regression

In [ ]:
lr_model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000, C=0.85)
lr_model.fit(X_train, y_train)
y_val_pred_lr = lr_model.predict(X_val)
f1_lr = f1_score(y_val, y_val_pred_lr, average='macro')
print(f"Validation F1 Score with Logistic Regression (L1): {f1_lr}")


Validation F1 Score with Logistic Regression (L1): 0.8397190704007285


### Linear SVM

In [ ]:
l_svm_model = LinearSVC(max_iter=1000, C=0.8)

l_svm_model.fit(X_train, y_train)
y_val_pred_l_svm = l_svm_model.predict(X_val)
f1_l_svm = f1_score(y_val, y_val_pred_l_svm, average='macro')
print(f"Validation F1 Score with Linear SVM: {f1_l_svm}")

Validation F1 Score with Linear SVM: 0.8393944722472735


###  SVM

In [68]:
nl_svm_model = SVC(kernel='rbf', probability=True)

nl_svm_model.fit(X_train, y_train)
y_val_pred_nl_svm = nl_svm_model.predict(X_val)
f1_nl_svm = f1_score(y_val, y_val_pred_nl_svm, average='macro')
print(f"Validation F1 Score with Non-Linear SVM: {f1_nl_svm}")

Validation F1 Score with Non-Linear SVM: 0.8738038729601904


### XGBoost

In [69]:
xgb_model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',
    objective='multi:softmax',
    num_class=5,
    random_state=42
)

xgb_model.fit(X_train, y_train)
y_val_pred_xgb = xgb_model.predict(X_val)
f1_xgb = f1_score(y_val, y_val_pred_xgb, average='macro')
print(f"Validation F1 Score with XGBoost: {f1_xgb}")

Validation F1 Score with XGBoost: 0.8456556647623815


### HyperParameters tuning for LR

In [70]:
param_grid_lr = {
    'C': [0.08, 0.8, 8, 80],                
    'solver': ['liblinear', 'saga'],
    'penalty': ['l1', 'l2']
}

grid_search_lr = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_grid=param_grid_lr,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid_search_lr.fit(X_train, y_train)

best_params_lr = grid_search_lr.best_params_
print(f"Best Params for LR: {best_params_lr}")

best_lr_model = LogisticRegression( **best_params_lr, max_iter=1000)
best_lr_model.fit(X_train, y_train)
y_val_pred_best_lr = best_lr_model.predict(X_val)

f1_best_lr = f1_score(y_val, y_val_pred_best_lr, average='macro')
print(f"Tuned Validation F1 for LR: {f1_best_lr}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best Params for LR: {'C': 0.8, 'penalty': 'l1', 'solver': 'saga'}
Tuned Validation F1 for LR: 0.8269113143093904


### HyperParameters tuning for SVM

In [71]:

param_grid_nl_svm = {
    'kernel': ['rbf', 'poly', 'sigmoid'],  
    'C': [0.08, 0.8, 8, 80],                
    'gamma': ['scale', 'auto'],            
    'degree': [2, 3, 4]                   
}

nl_svm_model = SVC(probability=True)
grid_search_nl_svm = GridSearchCV(estimator=nl_svm_model, param_grid=param_grid_nl_svm, 
                               scoring='f1_macro', cv=3, verbose=2, n_jobs=-1)

grid_search_nl_svm.fit(X_train, y_train)

best_params_nl_svm = grid_search_nl_svm.best_params_
print(f"Best Params for SVM: {best_params_nl_svm}")

best_nl_svm_model = SVC(**best_params_nl_svm, probability=True)
best_nl_svm_model.fit(X_train, y_train)

y_val_pred_best_nl_svm = best_nl_svm_model.predict(X_val)
f1_best_nl_svm = f1_score(y_val, y_val_pred_best_nl_svm, average='macro')
print(f"Tuned Validation F1 for SVM: {f1_best_nl_svm}")


Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best Params for SVM: {'C': 8, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Tuned Validation F1 for SVM: 0.8492609794844483


### HyperParameters tuning for XGBoost

In [72]:
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 9, 11],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.2, 0.3],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [1, 1.5, 2],
}

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, 
                                   scoring='f1_macro', n_iter=20, cv=3, verbose=2, n_jobs=-1, random_state=42)

random_search.fit(X_train, y_train)

best_params = random_search.best_params_
print(f"Best Params for xgb: {best_params}")

best_xgb_model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
best_xgb_model.fit(X_train, y_train)

y_val_pred = best_xgb_model.predict(X_val)

f1_best_xgb = f1_score(y_val, y_val_pred, average='macro')
print(f"Tuned Validation F1 Score: {f1_best_xgb}")


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Params for xgb: {'subsample': 0.9, 'reg_lambda': 1, 'reg_alpha': 0.1, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 11, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.9}
Tuned Validation F1 Score: 0.8522157815302978


## all toghether

In [73]:
model_performance = pd.DataFrame({
    'Model': ['Naive Bayes', 'Logistic Regression', 'Linear SVM', 'Non-Linear SVM', 'XGBoost', 'Tuned Logistic Regression', 'Tuned Non-Linear SVM', 'Tuned XGBoost'],
    'F1 Score': [f1_nb, f1_lr, f1_l_svm, f1_nl_svm, f1_xgb, f1_best_lr, f1_best_nl_svm, f1_best_xgb]
})

model_performance.sort_values(by='F1 Score', ascending=False)

,Model,F1 Score
3,Non-Linear SVM,0.873804
7,Tuned XGBoost,0.852216
6,Tuned Non-Linear SVM,0.849261
4,XGBoost,0.845656
1,Logistic Regression,0.839719
2,Linear SVM,0.839394
5,Tuned Logistic Regression,0.826911
0,Naive Bayes,0.754195


### ENSEMBLE

In [77]:
ensemble_model = VotingClassifier(
    estimators=[
        ('tuned nl-svm', best_nl_svm_model),
        ('tuned xgb', best_xgb_model),
    ],
    voting='soft'
)

ensemble_model.fit(X_train, y_train)
y_val_pred_ensemble = ensemble_model.predict(X_val)

f1_ensemble = f1_score(y_val, y_val_pred_ensemble, average='macro')
print(f"Validation F1 Score with Ensemble Model: {f1_ensemble}")

Validation F1 Score with Ensemble Model: 0.8552901925579806


In [78]:
# Combine training and validation sets
X_full_train = X_selected
y_full_train = y

# Retrain the final model
final_model = ensemble_model
final_model.fit(X_full_train, y_full_train)


VotingClassifier(estimators=[('tuned nl-svm',
                              SVC(C=8, degree=2, probability=True)),
                             ('tuned xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.9, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='mlogloss',
                                            feature_types=None, gamma=0.2,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.1, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=11,
                                            max_leaves=None, min_child_weight=1,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='multi:softprob', ...))],
                 voting='soft')

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    پیش‌بینی مدل خود بر روی داده‌های آزمایش را در قالب یک <code>dataframe</code> ذخیره کنید.  این <code>dataframe</code> باید دارای یک ستون با نام <code>author</code> باشد که ردیف <code>i</code>ام آن، پیش‌بینی شما برای سطر <code>i</code>ام مجموعه‌داده‌ی آزمون باشد.
</font>
</p>

<div dir=rtl style="direction: rtl;text-align: center;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
|ستون|توضیحات|
|------|---|
|author|نویسنده‌ی متن|
    
</font>
</div>

In [79]:
test_predictions = final_model.predict(X_test_selected)
test_predictions_decoded = label_encoder.inverse_transform(test_predictions)
submission = pd.DataFrame(test_predictions_decoded, columns=['author'])
submission.to_csv('submission.csv', index=False)

download it

In [ ]:
from google.colab import files
submission.to_csv('submission.csv', index=False)
files.download('submission.csv')